In [ ]:
from generative_ecg.dataset import load_signals, get_peaks, segment, filter_beats, project, save_beat_dataset, load_beat_dataset
from generative_ecg.train import train_cnn
from generative_ecg.train import train_vae
# from generative_ecg.generate.generate_ecg import generate_and_save_ecgs

from pathlib import Path

ptb_xl_path = r"C:\Users\Aaron Zhang\Desktop\ecg_project\generative_ecg\examples\ptb_xl\data"
result_path = r"C:\Users\Aaron Zhang\Desktop\ecg_project\generative_ecg\examples\ptb_xl\results"

x_signals, y_signals = load_signals(filepath=ptb_xl_path, sampling_rate=500)
x_beats, y_beats = [], []

# TESTING
import jax
jax.numpy.save(result_path + r"\unprocessed\x_signals.npy", x_signals)
jax.numpy.save(result_path + r"\unprocessed\y_signals.npy", y_signals)

Loading data from records: 100%|██████████| 21799/21799 [21:38<00:00, 16.79it/s]   


# Generative ECG Example

In [4]:
from generative_ecg.dataset import load_signals, get_peaks, segment, filter_beats, project, save_beat_dataset, load_beat_dataset
from generative_ecg.train import train_cnn
from generative_ecg.train import train_vae
# from generative_ecg.generate.generate_ecg import generate_and_save_ecgs

from pathlib import Path
import tqdm
import jax

ptb_xl_path = r"C:\Users\Aaron Zhang\Desktop\ecg_project\generative_ecg\examples\ptb_xl\data"
result_path = r"C:\Users\Aaron Zhang\Desktop\ecg_project\generative_ecg\examples\ptb_xl\results"
x_signals = jax.numpy.load(result_path + r"\unprocessed\x_signals.npy")
y_signals = jax.numpy.load(result_path + r"\unprocessed\y_signals.npy")

x_beats, y_beats = [], []

for i, x_signal in enumerate(tqdm.tqdm(x_signals[:500], desc="Processing and Filtering Beats")):
    y_signal = y_signals[i]
    x_peaks = get_peaks(x_signal, sampling_rate=500)
    # TODO: ask whether process function has to use jax, because jax.numpy.array cannot take in variable len
    x_seg, x_windows = segment(x_signal, x_peaks, tmax=400)
    x_filter, y_filter = filter_beats(x_seg, y_signal, x_windows, x_peaks, drop_first=True, drop_last=True, range_min=0.5, sd_min=0.06)
    
    x_proj, y_proj = project(x_filter, y_filter, tol=1e-6)
    x_beats += x_proj
    y_beats += y_proj

    # try:
    #     print(5)

    # except:
    #     continue

print(x_beats, y_beats)

# Finally, create a custom save & load pair of functions if you want to skip the above work
save_beat_dataset(x_beats, y_beats, filepath=result_path)

Processing and Filtering Beats:   0%|          | 0/500 [00:01<?, ?it/s]


TypeError: unsupported operand type(s) for +: 'list' and 'ArrayImpl'

In [1]:
from generative_ecg.models import ECGConv, rmse_loss
from generative_ecg.dataset import load_signals, get_peaks, segment, filter_beats, project, save_beat_dataset, load_beat_dataset
import optax
import sklearn

x_beats, y_beats = load_beat_dataset(filepath=result_path)

print(x_beats)

model = ECGConv(tmax=400, n_channels=12, n_layers_conv=2, n_layers_dense=2, n_outputs=4)
loss_fn = rmse_loss
lr_schedule = optax.warmup_cosine_decay_schedule(
    init_value=1e-3,
    peak_value=1e-2,
    warmup_steps=50,
    decay_steps=1000,
    end_value=1e-3
)

X_tr, X_te, y_tr, y_te = sklearn.model_selection.train_test_split(x_beats, y_beats, random_state=42)

state = gen_ecg.train.train_cnn(X_tr, X_te, y_tr, y_te, model, loss_fn, lr_schedule, ckpt_dir=result_path, batch_size=64)


KeyboardInterrupt



In [ ]:
# del y_tr, X_te, y_te
result_path = Path(result_path)
result = train_vae(X_tr, ckpt_dir, result_path, beat_segment=True, processed=True, n_epochs=5)

In [ ]:
generate_and_save_ecgs(X_tr, result, result_path, processed=True)